In [ ]:
import pandas as pd
import numpy as np

In [ ]:
column_names = ['id', 'series', 'date', 'query', 'user_id', 'twitters']

df_twitters = pd.read_csv('/home/borges/Documentos/Bases_dados/twitter/training.1600000.processed.noemoticon.csv',
                          encoding='latin1', names=column_names)

In [ ]:
print(df_twitters.shape)
df_twitters.head(10)

# Subir os dados para o banco de dados

In [ ]:
!pip install mysql-connector-python

In [ ]:

import mysql.connector

def criar_conexao(host, usuario, password, banco):
    return mysql.connector.connect(
        host=host,
        user=usuario,
        password=password,
        database=banco
    )


def fechar_conexao(con):
    return con.close()




In [ ]:
# criar tabela se não existir

mydb = criar_conexao("localhost", "root", "essaeasenha", "twitter")

mycursor = mydb.cursor()

mycursor.execute("""CREATE TABLE IF NOT EXISTS tweets (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    id_tweets BIGINT,
                    series VARCHAR(255),
                    date VARCHAR(100),
                    query VARCHAR(100),
                    user_id VARCHAR(255),
                    twitters VARCHAR(255)
                    );
                """)

In [ ]:
# criando conexção como o banco de dados
conn = criar_conexao("localhost", "root", "essaeasenha", "twitter")
cursor = conn.cursor()

# dividit o dataframe em partes
df_splits = np.array_split(df_twitters.sort_values(by=['series']), 100)

count_batches = 1

for df_part in df_splits:
      for index, row in df_part.iterrows():
            id_tweets = row[0]
            series = row[1]
            date = row[2]
            query = row[3]
            user_id = row[4]
            twitters = row[5]
            
            sql = """INSERT INTO tweets (
             id_tweets, series, date,
             query, user_id, twitters) VALUES (
             %s, %s, %s, %s, %s, %s)
            """
            valores = (id_tweets, series, date,
                        query, user_id, twitters)
            
            cursor.execute(sql, valores)
            
            
      conn.commit()
      cursor.close()
      cursor = conn.cursor()
      count_batches += 1
      
      print(f'=====> LOTE {count_batches} SUBIU <===== \n')

# faz mais um commit para atualizar as linhas restantes
conn.commit()

# imprime a quantidade de lotes atualizados
print(f'{count_batches} lotes atualizados.')

fechar_conexao()
